<a href="https://colab.research.google.com/github/lepremiere/Data-Collection/blob/main/000_Tutorial/03_Practical_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Practical Example**



In [ ]:
import numpy as np

In [ ]:
def I(n, m):
  return np.eye(n, m, dtype=np.float32)

In [ ]:
R0 = np.array([2, 3, 4], dtype=np.float32)
R1 = np.array([1, 4, 2], dtype=np.float32)

R = np.stack([R0, R1], axis=1)
G = 1/R
E = np.ones((2, 1)).astype(np.float32)

# Incoming wave
a = np.array([[0.5, 0.25, 0.1], [0.3, 0.7, 0.9]], dtype=np.float32).T

In [ ]:
Rx = np.kron(I(2, 2), I(3, 3)) * R.reshape(-1, order="F")
Gx = np.kron(I(2, 2), I(3, 3)) * G.reshape(-1, order="F")
Ex = np.kron(E, I(3, 3))

**Serial Adaptor**

In [ ]:
def Gamma_T_kron(G, E):
  return 2 * np.linalg.inv(E.T @ G @ E) @ E.T @ G

def Gamma_T_einsum(G, E):
  return 2 * np.einsum("ij,ij->ij", 1/(G @ E), G) # Equivalent to: 2 * G/(G @ E)

In [ ]:
gamma_T_kron = Gamma_T_kron(Gx, Ex)
gamma_T_einsum = Gamma_T_einsum(G, E)

print("Gamma.T (Kron): \n", gamma_T_kron)
print("Gamma.T (Einsum):\n", gamma_T_einsum)

assert (gamma_T_kron.sum(axis=1) == gamma_T_einsum.sum(axis=1)).all()

Gamma.T (Kron): 
 [[0.6666667 0.        0.        1.3333334 0.        0.       ]
 [0.        1.1428571 0.        0.        0.8571428 0.       ]
 [0.        0.        0.6666667 0.        0.        1.3333334]]
Gamma.T (Einsum):
 [[0.6666667 1.3333334]
 [1.1428571 0.8571428]
 [0.6666667 1.3333334]]


In [ ]:
# Kron
ax = a.reshape(-1, order="F")
EI = np.kron(np.ones((2,2)), I(3,3)).astype(np.float32)
Sx = Ex @ gamma_T_kron - EI
bx = Sx @ ax

# Einsum
S = (gamma_T_einsum - 1)
b = np.einsum("i,ij->ij", np.einsum("ij,ij->i", S, a), np.ones_like(a))

# print(S)
# print(Sx)
# print(a)
print("b (Kron): \n", bx)
print("b (Einsum): \n", b)

assert bx.sum() == b.sum()

b (Kron): 
 [-0.06666664 -0.06428576  0.2666667  -0.06666664 -0.06428576  0.2666667 ]
b (Einsum): 
 [[-0.06666664 -0.06666664]
 [-0.06428576 -0.06428576]
 [ 0.2666667   0.2666667 ]]


**Parallel Adaptor**

In [ ]:
def Gamma_kron(R, E):
  return 2 * R @ E @ np.linalg.inv(E.T @ R @ E)

def Gamma_einsum(R, E):
  return 2 * np.einsum("ij,ij->ij", R, 1/(R @ E)) # Equivalent to: 2 * R/(R @ E)

In [ ]:
gamma_kron = Gamma_kron(Rx, Ex)
gamma_einsum = Gamma_einsum(R, E)

print("Gamma (Kron): \n", gamma_kron)
print("Gamma (Einsum):\n", gamma_einsum)

assert (gamma_kron.T.sum(axis=1) == gamma_einsum.sum(axis=1)).all()

Gamma (Kron): 
 [[1.3333334 0.        0.       ]
 [0.        0.8571429 0.       ]
 [0.        0.        1.3333334]
 [0.6666667 0.        0.       ]
 [0.        1.1428572 0.       ]
 [0.        0.        0.6666667]]
Gamma (Einsum):
 [[1.3333334 0.6666667]
 [0.8571429 1.1428572]
 [1.3333334 0.6666667]]


In [ ]:
# Kron
Sx = EI - gamma_kron @ Ex.T
bx = Sx @ ax

# Einsum
S = 1. - gamma_einsum 
b = np.einsum("ij,ik->ij", S, a)

print(S)
print(Sx)
print(a)
print("b (Kron): \n", bx)
print("b (Einsum): \n", b)

print(bx.sum() , b.sum())

[[-0.33333337  0.3333333 ]
 [ 0.14285707 -0.1428572 ]
 [-0.33333337  0.3333333 ]]
[[-0.33333337  0.          0.         -0.33333337  0.          0.        ]
 [ 0.          0.14285707  0.          0.          0.14285707  0.        ]
 [ 0.          0.         -0.33333337  0.          0.         -0.33333337]
 [ 0.3333333   0.          0.          0.3333333   0.          0.        ]
 [ 0.         -0.1428572   0.          0.         -0.1428572   0.        ]
 [ 0.          0.          0.3333333   0.          0.          0.3333333 ]]
[[0.5  0.3 ]
 [0.25 0.7 ]
 [0.1  0.9 ]]
b (Kron): 
 [-0.2666667   0.13571422 -0.33333337  0.26666665 -0.13571432  0.3333333 ]
b (Einsum): 
 [[-0.2666667   0.26666665]
 [ 0.13571422 -0.13571432]
 [-0.33333337  0.3333333 ]]
-2.0861626e-07 -2.3841858e-07
